In [2]:
import tensorflow as tf

from utils.helper_utils import HelperUtils
from configuration import Configuration
from utils.testing_util import TestingUtil

In [3]:
data_dir = HelperUtils.load_data(f'{Configuration.data_location}/{Configuration.model_name}.zip', f'{Configuration.model_name}_dataset')
HelperUtils.convert_dataset(data_dir)

Loading data from file:///C:\Users\thehe\dev\studies\iui\iui\data\updated.zip
132128768/132116973 [==============================] - 0s 0us/step


In [4]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=Configuration.validation_split,
    subset="training",
    seed=123,
    image_size=(Configuration.img_height, Configuration.img_width),
    batch_size=Configuration.batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=Configuration.validation_split,
    subset="validation",
    seed=123,
    image_size=(Configuration.img_height, Configuration.img_width),
    batch_size=Configuration.batch_size)

class_names = train_ds.class_names

num_classes = len(class_names)

# train_ds = train_ds.shuffle(500).batch(batch_size)
# val_ds = val_ds.shuffle(500).batch(batch_size)

Found 86034 files belonging to 440 classes.
Using 68828 files for training.
Found 86034 files belonging to 440 classes.
Using 17206 files for validation.


In [10]:
model,  prev_class_names = HelperUtils.load_model(Configuration.existing_model_name)
# opt = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9)
# model.compile(
#     optimizer='adam',
#     loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#     metrics=['accuracy']
# )
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 180, 180, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 180, 180, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 90, 90, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 90, 90, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 45, 45, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 45, 45, 64)        1

In [11]:
model.fit(train_ds, validation_data=val_ds, epochs=Configuration.epochs)
model.save(f'{Configuration.models_location}/{Configuration.model_name}.h5')

Epoch 1/10
 150/2151 [=>............................] - ETA: 31s - loss: nan - accuracy: 0.0092

KeyboardInterrupt: 

In [7]:
TestingUtil.test_with_test_set(model, class_names)

24576/15169 [================================================] - 0s 0s/step
                                             image   class prediction Correct
0         c0_1_P43_X6xu_original_1618572393300.jpg    2420      10197      No
1                   P43_56dz_P43_1618572375827.jpg    2420      10197      No
2                   P43_TDGu_P43_1618572365712.jpg    2420      10197      No
3                     B2_nMPI_B2_1618996575203.jpg   43713      10197      No
4                     B2_W9U4_B2_1618996605608.jpg   43713      10197      No
5          c0_2_P8_RUY6_original_1619088491932.jpg   43719      10197      No
6                     P8_cGM2_P8_1619088524002.jpg   43719      10197      No
7          c0_1_F6_ImhL_original_1619600612424.jpg   43898      10197      No
8          c2_2_F6_bcU4_original_1619600733320.jpg   43898      10197      No
9                     F6_dCiY_F6_1619600709872.jpg   43898      10197      No
10                  B15_jgxl_B15_1618305884455.jpg   4460b      10